In [1]:
from importlib import reload
import numpy as np
import time

In [2]:
from ics.cobraCharmer import pfi as pfiControl
reload(pfiControl)

<module 'ics.cobraCharmer.pfi' from '/home/cwen/mhs/devel/ics_cobraCharmer/python/ics/cobraCharmer/pfi.py'>

In [3]:
mod1Cobras = pfiControl.PFI.allocateCobraRange(range(1,2))
allCobras = mod1Cobras
oneCobra = pfiControl.PFI.allocateCobraList([(1,2)])
twoCobras = pfiControl.PFI.allocateCobraList([(1,2), (1,5)])

# partition module 1 cobras into non-interfering sets
moduleCobras = {}
for group in 1,2,3:
    cm = range(group,58,3)
    mod = [1]*len(cm)
    moduleCobras[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
group1Cobras = moduleCobras[1]
group2Cobras = moduleCobras[2]
group3Cobras = moduleCobras[3]

# partition module 1 cobras into odd and even sets
moduleCobras2 = {}
for group in 1,2:
    cm = range(group,58,2)
    mod = [1]*len(cm)
    moduleCobras2[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
oddCobras = moduleCobras2[1]
evenCobras = moduleCobras2[2]

In [4]:
#pfi = pfiControl.PFI(fpgaHost='128.149.77.24') #'fpga' for real device.
pfi = pfiControl.PFI(fpgaHost='localhost', doLoadModel=False)
pfi.loadModel('../xml/updatedLinksAndMaps.xml')
pfi.setFreq(allCobras)

2018-10-22T17:33:31.151 20 fpgaIO     FPGA connection to localhost
2018-10-22T17:33:31.611 20 fpga       load cobra model from ../xml/updatedLinksAndMaps.xml
2018-10-22T17:33:31.613 20 log.short  --- ISSUE SETFREQ & VERIFY (brd:1) ---
2018-10-22T17:33:31.615 20 log.eth    (ETH)Sent msg on socket.
(0300003907d088cd	081300fd00940823	00fd0099101300fb	009a102300fc0099	
181300fd00981823	00fc009720130100	0097202300fc0098	281300fa00992823	
00fc0097301300fe	0095302300fd0098	381300fe00973823	00fc0098401300fa	
0097402300fc009a	481300fc00984823	00fe0096501300fd	0098502300fa0097	
581300fb00975823	00fe0099601300fc	0099602300fa0098	681300fd00976823	
01000096701300fd	0099702300fd0094	781300fc00977823	00fc0098801300fb	
0093802300fb0096	881300fb00988823	00fd0098901300fd	0096902300fa0098	
981300ff00979823	00fa0098a01300ff	0099a02300fe0096	a81300fc0097a823	
01000098b01300fd	0097b02301010099	b81300fa0097b823	00fc0098c01300fa	
0097c02300fe0098	c81300fc0096c823	00f80096d01300fa	0097d02300f90097	
d81300f8009

In [5]:
pfi.loadModel('../xml/ASRD1016.xml')

2018-10-22T17:33:31.803 20 fpga       load cobra model from ../xml/ASRD1016.xml


In [ ]:
pfi.disconnect()

In [ ]:
pfi.connect(fpgaHost='localhost')

In [ ]:
pfi.reset()

In [ ]:
pfi.power()

In [ ]:
pfi.moveThetaPhi(oneCobra, [-0.1], [-0.1], [1.0], [0.5])

In [ ]:
pfi.moveAllThetaPhi(allCobras, 0.1, 0.2, thetaHome='ccw', phiHome='CW')

In [ ]:
pfi.thetaToGlobal(oddCobras, [np.deg2rad(1)]*len(oddCobras), thetaHome=pfi.CW)

In [ ]:
pfi.thetaToLocal(oddCobras, [np.deg2rad(15)]*len(oddCobras), thetaHome='cw')

In [ ]:
# Calculate outward angles
oddMoves = pfi.thetaToLocal(oddCobras, [np.deg2rad(135)]*len(oddCobras))
oddMoves[oddMoves>1.85*np.pi] = 0

evenMoves = pfi.thetaToLocal(evenCobras, [np.deg2rad(315)]*len(evenCobras))
evenMoves[evenMoves>1.85*np.pi] = 0

allMoves = np.zeros(57)
allMoves[::2] = oddMoves
allMoves[1::2] = evenMoves

def moveCobra(c, theta, phi):
    pfi.moveSteps([allCobras[c-1]], np.zeros(1)+theta, np.zeros(1)+phi)

def moveCobras(cs, theta, phi):
    cobs = []
    for c in cs:
        cobs.append(allCobras[c-1])
    pfi.moveSteps(cobs, np.array(theta), np.array(phi))

In [ ]:
# The updated steps for moving cobras to up/down positions
allSteps = np.array([3200., 4300,    0., 4800., 1400,
 6000.,  1400., 3267.39039139, 2934.04991828, 0.,
 1440.0769006,  1108.36510939, 2447.07789072, 3300., 6000.,
   22.40497809,  233.60670712, 2846.80763061,  717.26507015, 1432.59642308,
  263.22585701, 1200.,  135.92568801, 4880.22316911,    0.,
 1700.,  411.15612259, 4190.63271838,  309.95326555,  937.61978123,
 3527.60958129, 1882.10623443,  667.63393703, 2331.33049693, 1008.7811692,
    0.,         3430.76739972, 2833.79313889,  836.87837639, 4200.,
 1389.91444697, 2214.89547244, 3659.6670764,  900., 3624.95167338,
 3200.,    0.,         4000.,  707.3192973,  2546.98893839,
 1919.88595392, 1753.70977189, 3894.01513043, 3600., 900.,
 1847.58659938,  833.76207202])

In [ ]:
# move a few cobras to outward position
cs = [28,31,38,41,46]
cind = np.array(cs) - 1
cnt = len(cind)

for i in range(3):
    # Home theta for multiple cobras
    moveCobras(cs, [-15000] * cnt, [0] * cnt)

    # Move multiple cobras
    moveCobras(cs, allSteps[cind], [0] * cnt)

    # sleep for three second
    time.sleep(3.0)

In [ ]:
# move all cobras to outward position
for i in range(3):
    # Home theta for all cobras
    pfi.moveSteps(allCobras, np.zeros(57)-15000, np.zeros(57))

    # Move all cobras to up/down positions
    pfi.moveSteps(allCobras, allSteps, np.zeros(57))

    # sleep for three second
    time.sleep(3.0)

In [ ]:
# Home theta
pfi.homeTheta(allCobras, nsteps=15000, dir='ccw')

In [ ]:
# Home phi in safe way
pfi.homePhiSafe(allCobras, nsteps=5000, dir='ccw')

In [ ]:
# Home phi
pfi.homePhi(allCobras, dir='ccw')

In [ ]:
# Move one cobra
moveCobra(4, 4800, 0)

In [ ]:
# Move multiple cobras
moveCobras((28, 46), (4800, 5200), (20, -100))

In [ ]:
# Move all cobras to up/down positions, the steps are calculated from motor maps
pfi.moveThetaPhi(allCobras, allMoves, np.zeros(57))

In [ ]:
# The angles to up/down positions
np.rad2deg(allMoves)

In [ ]:
# Cobra motor map parameters
print(pfi.calibModel.motorFreq1, pfi.calibModel.motorFreq2)
print(pfi.calibModel.motorOntimeFwd1, pfi.calibModel.motorOntimeFwd2)
print(pfi.calibModel.tht0, pfi.calibModel.tht1)

In [ ]:
# Script to measure cobra centers, arm lengths, motor maps
steps = 500
zeros = np.zeros(19)
for g, cobraGroup in enumerate(moduleCobras.values()):
    pfi.moveSteps(cobraGroup, zeros-15000, zeros)
    for i in range(10):
        pfi.moveSteps(cobraGroup, zeros, zeros+steps)
        time.sleep(0.5)
    pfi.moveSteps(cobraGroup, zeros, zeros+5000)
    time.sleep(5)
    for i in range(30):
        pfi.moveSteps(cobraGroup, zeros+steps, zeros)
        time.sleep(0.5)
    pfi.moveSteps(cobraGroup, zeros+15000, zeros)
    time.sleep(5)
    for i in range(30):
        pfi.moveSteps(cobraGroup, zeros-steps, zeros)
        time.sleep(0.5)
    pfi.moveSteps(cobraGroup, zeros-15000, zeros)
    time.sleep(5)
    for i in range(10):
        pfi.moveSteps(cobraGroup, zeros, zeros-steps)
        time.sleep(0.5)
    pfi.moveSteps(cobraGroup, zeros, zeros-5000)
    time.sleep(5)
    pfi.moveSteps(cobraGroup, allSteps[g::3], np.zeros(19))
    time.sleep(5)

In [ ]:
np.rad2deg(pfi.motorMap.phiOffsets[0])

In [ ]:
pfi.motorMap.posPhiSteps[0]

In [ ]:
pfi.motorMap.negPhiSteps[0]

In [ ]:
np.rad2deg(pfi.motorMap.phiOffsets[0])

In [ ]:
pfi.moveSteps(oneCobra, [15000], [-1000], dirs=[(-1,1)])

In [ ]:
pfi.anglesToPositions(oneCobra, [0], [0])

In [ ]:
pfi.anglesToPositions(twoCobras, [1.8, 0.01], [0.5, 3.0])

In [ ]:
pfi.anglesToPositions(twoCobras, [0.4, 0.01], [0.01, 1.0])

In [ ]:
pfi.positionsToAngles(twoCobras, [1711.41620152+971.24790363j, 1583.51743468+946.28961668j])

In [6]:
pfi.moveXY(twoCobras,
           [1711.04763067+971.6405348j, 1599.65210356+901.09703541j],
           [1711.41620152+971.24790363j, 1583.51743468+946.28961668j],
          )

2018-10-22T17:33:38.498 20 fpga       engaged cobras: [(1, 2), (1, 5)]
2018-10-22T17:33:38.500 20 fpga       move to: [(0.7782917764206472, 0.03805589509404259), (0.010000000777012685, 2.999999998622861)]
2018-10-22T17:33:38.501 20 fpga       move from: [(0.6321349466890247, 0.04805589558317806), (0.010000000036441525, 1.0000000000915832)]
2018-10-22T17:33:38.505 20 fpga       steps: [(208.55818989010788, -3.1110268944791297), (3.6375140943789575e-07, 855.0409560482854)]
2018-10-22T17:33:38.506 20 log.short  --- ISSUE RUN & VERIFY (2) ---
2018-10-22T17:33:38.507 20 log.full   (C2,1 (35, 30)uS x(208, 3) d(0, 0) ['cw', 'ccw']) (C1,3 (56, 38)uS x(0, 855) d(0, 0) ['cw', 'cw']) 
2018-10-22T17:33:38.507 20 log.medium Timeout:65535, inter:156
2018-10-22T17:33:38.508 20 log.eth    (ETH)Sent msg on socket.
(01000002ffff009c	fb31082e002300d0	0000001e00030000	1818003800000000	
002603570000)
2018-10-22T17:33:38.510 20 log.eth    (ETH)Rcvd msg on socket.
(010000000000)
2018-10-22T17:33:38.510 20 lo

In [8]:
pfi.moveXYfromHome(twoCobras, [1703.7830214 +963.56304167j, 2583.51743468+946.28961668j])

2018-10-22T17:33:55.102 20 fpga       some target positions are invalid
2018-10-22T17:33:55.104 20 fpga       engaged cobras: [(1, 2)]
2018-10-22T17:33:55.105 20 fpga       move to: [(1.7999999999445446, 0.4999999998734284)]
2018-10-22T17:33:55.107 20 fpga       move from: [(0.0, 0.0)]
2018-10-22T17:33:55.111 20 fpga       steps: [(2428.7516439294795, 249.88722659498183)]
2018-10-22T17:33:55.113 20 log.short  --- ISSUE RUN & VERIFY (2) ---
2018-10-22T17:33:55.113 20 log.full   (C2,1 (35, 21)uS x(2428, 249) d(0, 0) ['cw', 'cw']) 
2018-10-22T17:33:55.114 20 log.medium Timeout:65535, inter:156
2018-10-22T17:33:55.115 20 log.eth    (ETH)Sent msg on socket.
(01000001ffff009c	fb7a082c0023097c	0000001500f90000	)
2018-10-22T17:33:55.116 20 log.eth    (ETH)Rcvd msg on socket.
(010000000000)
2018-10-22T17:33:55.117 20 log.medium Run tlm rx'd. (Code:0) (Message:0)
2018-10-22T17:33:55.118 20 log.eth    (ETH)Rcvd msg on socket.
(010000000000)
2018-10-22T17:33:55.118 20 log.medium Run tlm rx'd. (Cod

In [ ]:
a = np.array([1.0, 3.0])
b = [2.0, 2.0]

In [ ]:
a < b

In [ ]:
np.any(np.all([a > b, a < b], axis=0))

In [ ]:
c = np.array([0.0, np.nan])

In [ ]:
[tht >= 0 for tht in c]

In [ ]:
np.isnan(c) == False

In [ ]:
np.nan < 0

In [ ]:
np.log(-1.0)